# Result Validation

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from gpytorch import settings
import torch
import torchvision.transforms as transforms
import gpytorch
import pandas as pd

In [3]:
DATASET_PATH = '/data/histopathologic-cancer-detection/'

In [4]:
model_type = 'resnet18'

# dataset = 'x_ray_binary'
dataset = 'cancer'

In [5]:
import json
def load_configuration(filename, path='./configuration'):
    with open(f'{path}/{filename}') as f:
        configuration = json.load(f)
    return configuration

In [6]:
config = load_configuration(filename=f'freq-{model_type}-{dataset}.json', path='./../deep_gp/configuration')
batch_size = 400
img_size = config['img_size']
n_channels = config['n_channels']

In [7]:
# define data transformations
common_trans = [
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6976, 0.5404, 0.6897],
                         std=[0.2433, 0.2839, 0.2199])
]

transform_test = transforms.Compose(common_trans)

use_cuda = torch.cuda.is_available()
torch.cuda.set_device(0)

In [8]:
from torchvision.models import ResNet
from torchvision.models.resnet import BasicBlock
import torch.utils.model_zoo as model_zoo
import math
import torch
from torch.distributions import Categorical

In [9]:
import os
from glob import glob
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torchvision.transforms import transforms
from PIL import Image


class HistoPathologicCancer(Dataset):
    """Histo-Pathologic Cancer Dataset with binary labeling: Finding/No-Finding"""

    def __init__(self, path=None, img_size=64, dataset_type='train', transform=None, n_channels=3, is_debug=False):
        self.path = path if path is not None \
            else '/home/alberto/Desktop/datasets/histopathologic-cancer-detection/'
        self.img_size = img_size
        self.dataset_type = dataset_type
        if self.dataset_type not in ['train', 'validation', 'test']:
            raise RuntimeError



        self.n_channels = n_channels
        self.is_debug = is_debug

        # define transformation
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                                transforms.Resize((img_size, img_size)),
                                transforms.ToTensor()])

        self.transform_target = transforms.ToTensor()

        self.df_data = pd.read_csv(os.path.join(self.path, 'train_labels.csv'))

        if is_debug:
            self.df_data = self.df_data[:512]

        # create training/validation set
        train, validation_and_test = train_test_split(self.df_data, random_state=42, shuffle=True, test_size=0.4)
        validation, test = train_test_split(validation_and_test, random_state=42, shuffle=True, test_size=0.5)

        if self.dataset_type == 'train':
            self.df_data = train
        elif self.dataset_type == 'validation':
            self.df_data = validation
        elif self.dataset_type == 'test':
            self.df_data = test
        else:
            raise ValueError

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        item = self.df_data.iloc[idx]
        img = self.get_rgb_image_from_file(os.path.join(self.path, 'train', ''.join([item['id'],
                                                                                     '.tif'])))
        img = self.transform(img)
        label = np.array(item['label'])
        label = torch.from_numpy(label)
        return img, label

    def load_image_from_file(self, filename):
        return Image.open(filename)

    def get_rgb_image_from_file(self, filename):
        return self.load_image_from_file(filename).convert('RGB')

    def get_grayscale_image_from_file(self, filename):
        return self.load_image_from_file(filename).convert('L')


In [10]:
test_set = HistoPathologicCancer(path=DATASET_PATH, img_size=img_size, dataset_type='test',
                                transform=transform_test,
                                is_debug=False)
num_classes = 2

In [11]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)

In [12]:
from torchvision.models import resnet18
from torch.nn import CrossEntropyLoss

In [24]:
model = resnet18(pretrained=False, num_classes=num_classes).cuda()
criterion = CrossEntropyLoss()

In [25]:
# load model
model_filename = f'./../deep_gp/frequentist-{model_type}-{dataset}.dat'
print(f'Loading model from {model_filename}')
trained_model = torch.load(model_filename)
model.load_state_dict(trained_model['model'])

Loading model from ./../deep_gp/frequentist-resnet18-cancer.dat


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1000, 512]) from checkpoint, the shape in current model is torch.Size([2, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([2]).

In [15]:
# set to eval mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

## Test

In [17]:
import torch.nn.functional as F

In [18]:
correct = 0
y_true = []
y_pred = []
fc_layer_result = []
softmax_layer_result = []

for data, target in test_loader:
    data, target = data.cuda(), target.cuda()
    with torch.no_grad():
        fc = model(data)
        fc_layer_result.append(fc.cpu().numpy())
        
        softmax = F.softmax(fc, dim=1)
        softmax_layer_result.append(softmax.cpu().numpy())
        
        _, predicted = torch.max(fc.data, 1)
        correct += float((predicted == target).sum().item())
        
        y_pred.extend(list(predicted.cpu().numpy()))
        y_true.extend(list(target.view_as(predicted).cpu().numpy()))
        
#         output, softmax, mixed_fs, samples = likelihood(model(data))
# #         plot_samples_batch(batch_samples=samples)
#         pred = output.probs.argmax(1).cpu()
#         y_pred.extend(list(pred.numpy()))
#         y_true += list(target.view_as(pred).cpu().numpy())
#         correct += float(pred.eq(target.view_as(pred).cpu()).sum())

accuracy = 100. * correct / float(len(test_loader.dataset))
print(f'Test set: Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy}%)')

Test set: Accuracy: 41924.0/44005 (95.27099193273492%)


In [23]:
fc.shape

torch.Size([5, 1000])

In [26]:
y_all = pd.DataFrame({'y_true': y_true,
                     'y_pred': y_pred})
y_all['residuals'] = y_all['y_true'] - y_all['y_pred']
# y_all.to_hdf('./test-results/y_all-resnet18.h5', 'y_all')
y_all.head()

,y_true,y_pred,residuals
0,0,0,0
1,0,0,0
2,0,0,0
3,1,1,0
4,0,0,0


In [28]:
y_all['y_pred'].unique()

array([0, 1])

In [19]:
fc_layer = np.concatenate(fc_layer_result, axis=0)
np.save(f'./network-output/fc_layer-Freq_Resnet18.npy', fc_layer)
fc_layer.shape

(44005, 1000)

In [20]:
softmax_layer = np.concatenate(softmax_layer_result, axis=0)
np.save(f'./network-output/softmax_layer-Freq_Resnet18.npy', softmax_layer)
softmax_layer.shape

(44005, 1000)

In [21]:
softmax.shape

torch.Size([5, 1000])